# Agent Insights experiments
Previous notebook was quite full, so here is a fresh one

### Tools for our agent
We had decided to give our agents the ability to
- Search for a query using the web
- Scrape a page to find out more info

In [2]:
# Scraping tool
from bs4 import BeautifulSoup
import requests

banned_sites = ["calendar.google.com", "researchgate.net"]

def scrape_page(url: str):
    if any(substring in url for substring in banned_sites):
        print("Skipping site: {}".format(url))
        return None
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive',
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        text = " ".join([t.get_text() for t in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])])
        return text.replace('|','')
    except requests.RequestException as e:
        print(f"Failed to fetch {url}. Error: {e}")
        return None

In [3]:
# SBert Summarizer
from summarizer.sbert import SBertSummarizer
summarizer = SBertSummarizer('paraphrase-MiniLM-L6-v2')

def summarize(text, num_sentences=3):
    return summarizer(text, num_sentences=num_sentences)

/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# Search tool
from typing import Any, List, Literal
import requests
import os

k: int = 5
gl: str = "us"
hl: str = "en"
tbs = None
num_sentences = 7
serper_api_key=os.environ.get("SERPER_API_KEY")
search_type: Literal["news", "search", "places", "images"] = "search"

def serper_search(
        search_term: str, search_type: str = "search", **kwargs: Any
    ) -> dict:
    headers = {
        "X-API-KEY": serper_api_key or "",
        "Content-Type": "application/json",
    }
    params = {
        "q": search_term,
        **{key: value for key, value in kwargs.items() if value is not None},
    }
    response = requests.post(
        f"https://google.serper.dev/{search_type}", headers=headers, params=params
    )
    response.raise_for_status()
    search_results = response.json()
    return search_results

def parse_snippets(results: dict) -> List[str]:
    result_key_for_type = {
        "news": "news",
        "places": "places",
        "images": "images",
        "search": "organic",
    }
    snippets = []
    if results.get("answerBox"):
        answer_box = results.get("answerBox", {})
        if answer_box.get("answer"):
            snippets.append(answer_box.get("answer"))
        elif answer_box.get("snippet"):
            snippets.append(answer_box.get("snippet").replace("\n", " "))
        elif answer_box.get("snippetHighlighted"):
            snippets.append(answer_box.get("snippetHighlighted"))

    if results.get("knowledgeGraph"):
        kg = results.get("knowledgeGraph", {})
        title = kg.get("title")
        entity_type = kg.get("type")
        if entity_type:
            snippets.append(f"{title}: {entity_type}.")
        description = kg.get("description")
        if description:
            snippets.append(description)
        for attribute, value in kg.get("attributes", {}).items():
            snippets.append(f"{title} {attribute}: {value}.")

    for result in results[result_key_for_type[search_type]][:k]:
        if "snippet" in result:
            page = scrape_page(result["link"])
            if page is None:
                snippets.append(f"Title: {result['title']}\nLink: {result['link']}\nSnippet: {result['snippet']}\n")
            else:
                summarized_page = summarize(page, num_sentences=num_sentences)
                if len(summarized_page) == 0:
                    summarized_page = "None"
                snippets.append(f"Title: {result['title']}\nLink: {result['link']}\nSnippet: {result['snippet']}\nSummarized Page: <p>{summarized_page}</p>")

    if len(snippets) == 0:
        return ["No good Google Search Result was found"]
    return snippets

def parse_results(results: dict) -> str:
        snippets = parse_snippets(results)
        results_string = ""
        for idx, val in enumerate(snippets):
            results_string += f"<result{idx}>\n{val}\n</result{idx}>\n\n"
        return results_string

def custom_search(query: str, **kwargs: Any):
    results = serper_search(
            search_term=query,
            gl=gl,
            hl=hl,
            num=k,
            tbs=tbs,
            search_type=search_type,
            **kwargs,
        )
    return parse_results(results)

In [68]:
custom_search("Statistics on AI taking over jobs in the US 2023")

/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The defa

"<result0>\nTitle: AI Replacing Jobs Statistics: The Impact on Employment in 2023 - SEO.ai\nLink: https://seo.ai/blog/ai-replacing-jobs-statistics\nSnippet: With projections that AI could potentially replace around 800 million jobs worldwide by 2030, it's no wonder the world is brimming with ...\nSummarized Page: <p>AI Replacing Jobs Statistics: The Impact on Employment in 2023 Discover the current impact of AI on employment and the future challenges ahead. As we delve into this critical topic, we focus on providing you with an informative and structured analysis that unveils both the current landscape and the looming future of AI's impact on human employment. However, the majority of businesses foresee computers replacing human tasks within 50 years, highlighting that while AI is a reality, it has not fully realized its potential for job replacement just yet. In the next chapters we'll present all the details on the currently available statistics of the impact of AI on human employmen

In [111]:
import aiohttp
import asyncio
k = 3

result_key_for_type = {
        "news": "news",
        "places": "places",
        "images": "images",
        "search": "organic",
    }

async def scrape_page_async(url: str, summarize_page = False, num_sentences = 3):
    """
    Based on your observations from the Search_Engine, if you want more details from
    a snippet for a non-PDF page, pass this the page's URL and the page's title to
    scrape the full page and retrieve the full contents of the page.
    """

    print("Parsing: {}".format(url))
    if any(substring in url for substring in banned_sites):
        print("Skipping site: {}".format(url))
        return None
    else:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive',
            }
            async with aiohttp.ClientSession() as session:
                async with session.get(url, headers=headers, timeout=30) as response:
                    response.raise_for_status()
                    content = await response.text()

                    soup = BeautifulSoup(content, 'html.parser')
                    text = " ".join([t.get_text() for t in soup.find_all(
                        ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])])
                    text = text.replace('|', '')

                    if summarize_page:
                        return summarize(text, num_sentences=num_sentences)
                    return text
        except Exception as e:
            print(f"Failed to fetch {url}. Error: {e}")
            return None


async def serper_search_async(
    search_term: str, search_type: str = "search", **kwargs: Any
) -> dict:
    headers = {
        "X-API-KEY": serper_api_key or "",
        "Content-Type": "application/json",
    }
    params = {
        "q": search_term,
        **{key: value for key, value in kwargs.items() if value is not None},
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(f"https://google.serper.dev/{search_type}", headers=headers, json=params) as response:
            response.raise_for_status()
            search_results = await response.json()
            return search_results


async def parse_snippets_async(results: dict, scrape_pages: bool = False, summarize_pages: bool = True, num_sentences: int = 3) -> List[str]:
    snippets = []
    if results.get("answerBox"):
        answer_box = results.get("answerBox", {})
        if answer_box.get("answer"):
            snippets.append(f"The answer is {answer_box.get('answer')}")
        elif answer_box.get("snippet"):
            snippets.append(f"The answer might be in the snippet: {answer_box.get('snippet')}")
        elif answer_box.get("snippetHighlighted"):
            snippets.append(f"The answer might be in the snippet: {answer_box.get('snippetHighlighted')}")

    if results.get("knowledgeGraph"):
        kg = results.get("knowledgeGraph", {})
        title = kg.get("title")
        entity_type = kg.get("type")
        if entity_type:
            snippets.append(f"Knowledge Graph Results: {title}: {entity_type}.")
        description = kg.get("description")
        if description:
            snippets.append(f"Knowledge Graph Results: {title}: {description}.")
        for attribute, value in kg.get("attributes", {}).items():
            snippets.append(f"Knowledge Graph Results: {title} {attribute}: {value}.")

    if scrape_pages:
        tasks = []
        for result in results[result_key_for_type[search_type]][:k]:
            task = asyncio.create_task(scrape_page_async(result["link"], summarize_page=summarize_pages, num_sentences=num_sentences))
            tasks.append(task)
        summarized_pages = await asyncio.gather(*tasks)
        for i, page in enumerate(summarized_pages):
            result = results[result_key_for_type[search_type]][i]
            if page:
                snippets.append(f"Title: {result.get('title', '')}\nSource:{result['link']}\nSnippet: {result.get('snippet', '')}\nSummarized Page: {page}")
            else:
                snippets.append(f"Title: {result.get('title', '')}\nSource:{result['link']}\nSnippet: {result.get('snippet', '')}\n")
    else:
        for result in results[result_key_for_type[search_type]][:k]:
            snippets.append(f"Title: {result.get('title', '')}\nSource:{result['link']}\nSnippet: {result.get('snippet', '')}\n")

    if len(snippets) == 0:
        return ["No good Google Search Result was found"]
    return snippets


async def parse_results_async(results: dict, scrape_pages: bool = False, summarize_pages: bool = True, num_sentences: int = 3) -> str:
    snippets = await parse_snippets_async(results, scrape_pages=scrape_pages, summarize_pages=summarize_pages, num_sentences=num_sentences)
    results_string = ""
    for idx, val in enumerate(snippets):
        results_string += f"Res {idx + 1}:\n{val}\n\n"
    return results_string


async def arun_search_tool_for_agents(query: str):
    results = await serper_search_async(
        search_term=query,
        gl=gl,
        hl=hl,
        num=k,
        tbs=tbs,
        search_type=search_type,
    )
    return await parse_results_async(results, scrape_pages=False)

In [112]:
print(await arun_search_tool_for_agents("Statistics on AI taking over jobs in the US 2023"))

Res 1:
Title: AI Replacing Jobs Statistics: The Impact on Employment in 2023 - SEO.ai
Source:https://seo.ai/blog/ai-replacing-jobs-statistics
Snippet: With projections that AI could potentially replace around 800 million jobs worldwide by 2030, it's no wonder the world is brimming with ...


Res 2:
Title: Which U.S. Workers Are More Exposed to AI on Their Jobs? - Pew Research Center
Source:https://www.pewresearch.org/social-trends/2023/07/26/which-u-s-workers-are-more-exposed-to-ai-on-their-jobs/
Snippet: In 2022, 19% of American workers were in jobs that are the most exposed to artificial intelligence, in which the most important activities ...


Res 3:
Title: Generative AI and the future of work in America - McKinsey
Source:https://www.mckinsey.com/mgi/our-research/generative-ai-and-the-future-of-work-in-america
Snippet: By 2030, we further estimate a 23 percent increase in the demand for STEM jobs. Although layoffs in the tech sector have been making headlines ...





### Preparing mock text input

This is how we will generate mock testing data for our experiments, we use transcripts from Lex Friedman's podcasts

In [4]:
# Way to generate a random test input using transcripts from Lex Fridman's podcast
# Make sure you have the transcripts downloaded in the folder lex_whisper_transcripts

import test_on_lex

transcripts = test_on_lex.load_lex_transcripts(random_n=10, transcript_folder="./lex_whisper_transcripts/", chunk_time_seconds=20)

import random
def generate_test_input():
    idx = random.randint(0, 10)
    key = list(transcripts.keys())[idx]
    transcript = transcripts[key]
    trans_idx = random.randint(10, len(transcript)-10)
    latest = transcript[trans_idx:trans_idx+7]
    prev_transcripts, curr_transcripts = str.join(",", list(latest[0:5])), latest[5]
    return prev_transcripts + "\n" + curr_transcripts

generate_test_input()

Processing episode_028_large...
Processing episode_308_large...
Processing episode_188_large...
Processing episode_223_large...
Processing episode_044_large...
Processing episode_256_large...
Processing episode_181_large...
Processing episode_021_large...
Processing episode_231_large...
Processing episode_288_large...


" So Swift came from, we were just finishing off the first version of C++ support in Clang. And C++ is a very formidable and very important language, but it's also ugly in lots of ways. And you can't influence C++ without thinking there has to be a better thing, right? And so I started working on Swift, again,, with no hope or ambition that would go anywhere, just let's see what could be done, let's play around with this thing. It was me in my spare time, not telling anybody about it, kind of a thing, and it made some good progress. I'm like, actually, it would make sense to do this. At the same time, I started talking with the senior VP of software at the time, a guy named Bertrand Serlet., And Bertrand was very encouraging. He was like, well, let's have fun, let's talk about this. And he was a little bit of a language guy, and so he helped guide some of the early work and encouraged me and got things off the ground. And eventually told my manager and told other people, and it started

### Agent setup

#### Insight prompt

In [ ]:
expert_agent_prompt_blueprint = """
"Convoscope" is a multi-agent system in which you are the {agent_name} agent. You are a highly skilled and highly intelligent expert {agent_name}.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert {agent_name}. Only provide insights that would come from your role as the {agent_name}.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what has already been said.
- "Convoscope": an intelligence augmentation tool running on user's smart glasses or on their laptop that they use during conversations to improve conversations. Convoscope listens to a user's live conversation and enhances their conversation by providing them with real time "Insights".

[Your Expertise: {agent_name}]

As the {agent_name} agent, you {agent_insight_type}.

[Your Tools]
You have access to tools, which you should utilize to help you generate highly valuable, insightful, contextually relevant insights.

Limit your usage of the Search_Engine tool to 1 times. Mention your number of usages of the tool in your thoughts.

[Example Insights]
Here are some example insights structure that you should aim to generate, a summary is given instead of the entire transcript for brevity. From these examples, you just need to learn how to structure a good insight:

{examples}

[Previously Generated Insights]
Here are some insights that had already been previously generated, if any, for this conversation. You MUST not repeat any of these insights:

{insights_history}


**Remember** You must carefully read the history of insights generated for this conversation and ensure that you do not repeat any of the insights that have already been generated. If you repeat an insight, you will be penalized. Even if you generate a new insight, but it is very similar to a previous insight, you will be penalized. If you are unsure if your insight is too similar to a previous insight you should not generate the insight.

[Your Task]
<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool fails to fulfill your request, don't run the exact same request on the same tool again. Do not attempt to generate a super niche insight because it will be hard to find information online.

In your initial thought, you should first come up with a plan to generate the "Insight". The plan should include:

{agent_plan}

The plan should include a final step to generate the insight. The "Insight" must {insight_num_words} words or less. The "Insight" can omit filler words or replace words with symbols or acronyms to shorten its length where possible. If you don't have a very valuable and useful "Insight" for any reason, simply specify your "Insight" as the string "null". 

Once you have the "Insight", extract the url of the most relevant reference source used to generate this "Insight". Also, relay the motive of the why the insight benefits the conversation, with quotes from the transcript. If the "Insight" is not up to par with the examples, we should hide it and just return "null" for the "Insight".

Here are more detailed formatting instructions
{format_instructions}

Remember, the insight needs to be {insight_num_words} words or less!
<Task end>

[Input Transcript]
<Transcript start>{conversation_transcript}<Transcript end>{final_command}
"""

#### [WIP] Shorter and faster insight prompt

In [127]:
expert_agent_prompt_blueprint = """
## General Context
"Convoscope" is a multi-agent system that reads live conversation transcripts and provides real time "Insights", which are short snippets of intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that aim to lead the user's conversation to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. 

### Your Expertise: {agent_name}
You are a highly skilled and intelligent {agent_name} expert agent in this system, responsible for generating a specialized "Insight".
As the {agent_name} agent, you {agent_insight_type}.

### Your Tools
You have access to tools, which you should utilize to help you generate "Insights". Limit your usage of the Search_Engine tool to 1 times.
If a tool fails to fulfill your request, don't run the exact same request on the same tool again, and just continue without it.

### Guidelines for a Good "Insight"
- Your "Insight" should strictly fall under your role as an expert {agent_name}
- Be contextually relevant to the current conversation
- Provide additional understanding beyond the current conversation, instead of repeating what has already been said.

### Example Insights
Here are some example "Insights" to help you learn the structure of a good "Insight". A summary is given instead of the entire transcript for brevity.
{examples}

## Task
Generate an "Insight" for the following conversation transcript. 
<Transcript start>{conversation_transcript}<Transcript end>

### Additional Guidelines
- Do not attempt to generate a super niche insight because it will be hard to find information online.
- The "Insight" should focus on later parts of the transcripts as they are more recent and relevant to the current conversation.
- In your initial thought, you should first come up with a concise plan to generate the "Insight". The plan should include:
{agent_plan}. You are only able to make 1 quick action to generate the "Insight".
- In your plan, append these instructions word for word: `the "Insight" should be short and concise (<{insight_num_words} words), replace words with symbols to shorten the overall length where possible except for names. Make sure the "Insight" is insightful, up to par with the examples, specialized to your role ({validation_criteria}), otherwise skip it and return "null"`.

### Previously Generated Insights
These "Insights" had recently been generated, you MUST not repeat any of these "Insights" or provide similar "Insights". Generate a new "Insight" that is different from these "Insights":
{insights_history}

### Output
Once you have the "Insight", extract the url of the most relevant reference source used to generate this "Insight".
{format_instructions}

{final_command}
"""

In [128]:
expert_agent_config_list = {
    "Statistician": {
        "agent_name": "Statistician",
        "insight_num_words": 10,
        "agent_insight_type": """generate insights which focus on statistics, and quantitative data. Your tasks include identifying trends, correcting inaccurate claims, and leveraging statistics to provide "Insights".""",
        "agent_plan": """1. Come up with a general description of the "Insight" to generate. \n2.Identify the single most important quantitative data, statistics, etc. that is needed to generate the "Insight". Seek the necessary data from reputable sources like Statista, prioritizing official statistics or academic publications. """,
        "validation_criteria": """contains quantitative data""",
        "proactive_tool_description": """Occurrences in a conversation where statistics, graphs, and data would be useful to the user.""",
        "proactive_tool_example": """Conversation: Transcript compares the number of CS students in US and China.
Insight: US: 6% HS students in CS, China: <1% K-12 in programming""",
        "examples": """
1. Conversation: Transcript compares the number of CS students in US and China.
Insight: US: 6% HS students in CS, China: <1% K-12 in programming

2. Conversation: Transcript mentions "Should we ban plastic straws?".
Insight: 500mil straws in the US/day, 8.3bil straws pollute the world's beaches

3. Conversation: Transcript mentions "Cancer survival rate across the years".
Insight: Cancer survival rate: 49% in mid-70s to 68% now
""",
    },
    "FactChecker": {
        "agent_name": "FactChecker",
        "insight_num_words": 10,
        "agent_insight_type": """fact check any falsy claims made during a conversation. Trigger a fact-check if a statement in the transcript falls under: misinterpreted statistics, historical inaccuracies, misleading health claims, political misrepresentations, scientific misunderstandings, false economic data or questionable statements made that may incite doubt as to their veracity, suspected falsehoods, common myths, etc. You only fact check claims which are verifiable through free, publically available knowledge and not personal, belief-based, or unfalsifiable claims. Your response should be a fact check, not a factoid. If there is not a clear, distinct, clear-cut thing that fits the previous requirements to fact check, then just output "null".""",
        "agent_plan": """1. Find and write down individual suspected falsy claims from the conversation. Do not consider personal, belief-based, or unfalsifiable claims. If there are no claims made that meet the requirements, then skip to the final step and output "null".\n2. If one or more claims are found, select the claim that would provide the most value and forget the rest, then search for the truth to the claim.\n3. Generate the "Insight".""",
        "validation_criteria": """debunks the falsy claim, and provides brief elaboration""",
        "proactive_tool_description": """Trigger a fact-check if a statement in the transcript falls under: misinterpreted statistics, historical inaccuracies, misleading health claims, political misrepresentations, scientific misunderstandings, or false economic data. Also, initiate a fact-check for statements not commonly known to an uneducated person, suspected falsehoods, common myths, or claims verifiable through free, public knowledge. Do not consider personal, belief-based, or unfalsifiable claims.""",
        "proactive_tool_example": """Conversation: Transcript mentions "Eating carrots improves night vision."
 Insight: Carrots have vitamin A; don't grant night vision. WWII myth origin""",
        "examples": """
 1. Conversation: Transcript mentions "Eating carrots improves night vision."
 Insight: Carrots have vitamin A; don't grant night vision. WWII myth origin

 2. Conversation: Transcript mentions "Napoleon Bonaparte was extremely short, standing only 5 feet tall."
 Insight: Napoleon was 5'7"; average height, Misconception from French units

 3. Conversation: Transcript mentions "Humans only use 10% of their brains."
 Insight: Humans use 100% of their brains; brain imaging shows activity
 """,
    },
    "DevilsAdvocate": {
        "agent_name": "DevilsAdvocate",
        "insight_num_words": 12,
        "agent_insight_type": """assess the point of view being taken in the conversation and steel-man a contrary position. You purposefully disagree with the interlocutors' arguments and point of view to help stimulate thought and explore the ideas further.""",
        "agent_plan": """1. Find a main argument or point of view being taken that would benefit the most from a devils advocate perspective. Write down the original position. If no position/argument is found, skip to the final step and output "null".\n2. Think of insightful perspectives and generate a devil's advocate.""",
        "validation_criteria": """gives an interesting perspective but not too controversial""",
        "proactive_tool_description": """When it would be useful for the user to see a devil's advocate opinion (a steel-man argument supporting a viewpoint different from their own).""",
        "proactive_tool_example": """Conversation: Transcript mentions "Climate change is a hoax."
Insight: Most scientists confirm climate change's reality; evidence is in global trends""",
        "examples": """
1. Conversation: Transcript mentions "Climate change is a hoax."
Insight: Most scientists confirm climate change's reality; evidence is in global trends

2. Conversation: Transcript mentions "Vaccines cause autism".
Insight: Numerous studies show no vaccine-autism link; vaccines prevent disease outbreaks

3. Conversation: Transcript mentions "Artificial intelligence will replace all human jobs."
Insight: AI will create new jobs and industries, not just replace old ones
""",
    },
}

In [129]:
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class AgentInsight(BaseModel):
        """
        Query for an insight generation process
        """
        agent_insight: str = Field(
             description="the short insight generated by the agent")
        reference_source: str = Field(
             description="the most useful source used to generate this insight", default="")

agent_insight_parser = PydanticOutputParser(pydantic_object=AgentInsight)

def format_list_data(list_data: list):
    return "\n".join([f"{i+1}. {example}" for i, example in enumerate(list_data)])

def expert_agent_prompt_maker(expert_agent_config, conversation_transcript, insights_history: list = [], final_command=""):
    # Populating the blueprint string with values from the agent_config dictionary
    if final_command != "":
        final_command = "\n\n" + final_command
    expert_agent_prompt = expert_agent_prompt_blueprint.format(**expert_agent_config, final_command=final_command, conversation_transcript=conversation_transcript, insights_history=format_list_data(insights_history), format_instructions=agent_insight_parser.get_format_instructions())
    return expert_agent_prompt

In [130]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

open_ai_api_key = os.environ.get("OPEN_AI_API_KEY")

llm = ChatOpenAI(temperature=0, openai_api_key=open_ai_api_key, model="gpt-4-1106-preview")

class SearchInput(BaseModel):
    query: str = Field(description="a search query")

agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            coroutine=arun_search_tool_for_agents,
            description="Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events. It's a tool for understanding, navigating, and engaging with the digital world's vast knowledge.",
            args_schema=SearchInput
        ),
    ], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=2, early_stopping_method="generate", verbose=True)

In [131]:
# test_transcript = generate_test_input()
print(test_transcript)
agent_key = expert_agent_config_list["FactChecker"]
agent_prompt = expert_agent_prompt_maker(agent_key, test_transcript, [])
# print(agent_prompt)
await agent.arun(agent_prompt)

 And if you look at the progress of things, it's getting better and better and better. I mean, if you, you know. Yeah. We're a little bit of a stalemate here. Democracy's really blossomed between World War II and the year 2000, especially in the eighties and nineties, you had an incredible wave of fall, you know, where many, many authoritarian, regimes fell and were replaced by democracies. I think around 2015, the acceleration kind of came to a standstill a little bit. There's some good news in some countries and there's bad news in others. Like in the last 10 years, you've had, for example, the Philippines has gone backwards., Thailand has gone backwards. Bangladesh has gone backwards. Turkey has gone backwards. That's like a half billion people right there. So you've had some positives, like, you know, there was positive movement forward in Armenia, Malaysia, some other countries, but we're kind of at a stalemate right now. And what most people fear about where we are right now, who

{'agent_insight': 'null',
 'reference_source': 'https://www.idea.int/gsod-2021/global-report'}

In [62]:
agent2 = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events. It's a tool for understanding, navigating, and engaging with the digital world's vast knowledge.",
            args_schema=SearchInput
        ),
    ], llm, agent=AgentType.OPENAI_FUNCTIONS, max_iterations=2, early_stopping_method="generate", verbose=True)

In [132]:
# test_transcript = generate_test_input()
print(test_transcript)
agent_key = expert_agent_config_list["Statistician"]
agent_prompt = expert_agent_prompt_maker(agent_key, test_transcript, [])
# print(agent_prompt)
agent.run(agent_prompt)

 And if you look at the progress of things, it's getting better and better and better. I mean, if you, you know. Yeah. We're a little bit of a stalemate here. Democracy's really blossomed between World War II and the year 2000, especially in the eighties and nineties, you had an incredible wave of fall, you know, where many, many authoritarian, regimes fell and were replaced by democracies. I think around 2015, the acceleration kind of came to a standstill a little bit. There's some good news in some countries and there's bad news in others. Like in the last 10 years, you've had, for example, the Philippines has gone backwards., Thailand has gone backwards. Bangladesh has gone backwards. Turkey has gone backwards. That's like a half billion people right there. So you've had some positives, like, you know, there was positive movement forward in Armenia, Malaysia, some other countries, but we're kind of at a stalemate right now. And what most people fear about where we are right now, who

/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/mambaforge/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



Observation: <result0>
Title: The Global State of Democracy 2023: The New Checks and Balances | International IDEA
Link: https://www.idea.int/publications/catalogue/global-state-democracy-2023-new-checks-and-balances
Snippet: The global state of democracy in 2023 is complex, fluid and unequal. Across every region of the world, democracy has continued to contract, ...
Summarized Page: <p>Main navigation Main navigation Explore menu Breadcrumb The Global State of Democracy 2023 The global state of democracy in 2023 is complex, fluid and unequal. Indeed, while The Global State of Democracy 2023 shows some declines in countries that had been thought to be healthy democracies, at the same time there were encouraging improvements in countries where the level of oppression has been constant for years. Countervailing institutions include relatively new entities, such as human rights organizations and electoral management bodies, as well as civil society networks, popular movements and investi

{'agent_insight': '2023: Democracy scores ↓ in 11/29 countries',
 'reference_source': 'https://freedomhouse.org/report/nations-transit/2023/war-deepens-regional-divide/explore-data'}